In [3]:
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [4]:
kc = pd.read_csv('kc_house_data.csv')

In [5]:
kc.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [6]:
X = kc[['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade']]
Y = kc.price/kc.sqft_living

In [7]:
xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size=.2)

In [8]:
clf = tree.DecisionTreeRegressor(max_depth=10)
clf = clf.fit(xTrain, yTrain)

In [9]:
def get_tree_path(tree, feature_names, house):
    left = tree.tree_.children_left
    right = tree.tree_.children_right
    threshold = tree.tree_.threshold
    features  = [feature_names[i] for i in tree.tree_.feature]
    value = tree.tree_.value
    tree_path = []
    
    def recurse(left, right, threshold, features, node):
        if (threshold[node] != -2):
            if (house[features[node]] <= threshold[node]):
                tree_path.append(features[node] + " <= " + str(threshold[node]))
                recurse(left, right, threshold, features, left[node])
            else:
                tree_path.append(features[node] + " >= " + str(threshold[node]))
                recurse(left, right, threshold, features, right[node])
    
    recurse(left, right, threshold, features, 0)
    return tree_path

def path_to_query_string(path):
    qS = ''
    for condition in path:
        qS = qS + condition + ' and '
    qS = qS[:-5]
    return qS
    

In [10]:
clf.score(xTest, yTest)

0.2841034143872816

In [24]:
paths = []
for i, r in X.iterrows():
    paths.append(get_tree_path(clf, X.columns, r))
    
len(X.query(path_to_query_string(paths[0])))

26

In [9]:
# clf.predict([[2000, 3000, 2, 3]])